# Trained Model Interface
> "Why simple, when you can use design patterns?"

In [ ]:
# default_exp trained_model

In [ ]:
# hide
import blackhc.project.script
from nbdev.showdoc import *

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


In [ ]:
# exporti
from dataclasses import dataclass

import torch.nn

from batchbald_redux.consistent_mc_dropout import (
    BayesianModule,
    get_ensemble_predictions_labels,
)

In [ ]:
# exports


class TrainedModel:
    """Evaluate a trained model."""

    def get_log_probs_N_K_C_labels_N(self, pool_loader, device):
        raise NotImplementedError()

    def get_log_probs_N_K_C(self, loader, device):
        return self.get_log_probs_N_K_C_labels_N(loader, device)[0]


@dataclass
class TrainedMCDropoutModel(TrainedModel):
    num_samples: int
    model: BayesianModule

    def get_log_probs_N_K_C_labels_N(self, loader, device):
        log_probs_N_K_C, labels_B = self.model.get_predictions_labels(
            num_samples=self.num_samples,
            loader=loader,
            device=device,
        )

        return log_probs_N_K_C, labels_B


@dataclass
class TrainedEnsemble(TrainedModel):
    models: [torch.nn.Module]

    def get_log_probs_N_K_C_labels_N(self, pool_loader, device):
        log_probs_N_K_C, labels_B = get_ensemble_predictions_labels(
            modules=self.models, loader=pool_loader, device=device
        )
        return log_probs_N_K_C, labels_B